In [3]:
!pip install playwright



In [4]:
!playwright install

173.7 MiB [] 0% 0.0s173.7 MiB [] 0% 63.0s173.7 MiB [] 0% 48.6s173.7 MiB [] 0% 58.0s173.7 MiB [] 0% 31.3s173.7 MiB [] 0% 22.5s173.7 MiB [] 0% 18.0s173.7 MiB [] 0% 14.4s173.7 MiB [] 1% 11.5s173.7 MiB [] 1% 10.6s173.7 MiB [] 1% 9.8s173.7 MiB [] 1% 9.4s173.7 MiB [] 2% 9.2s173.7 MiB [] 2% 8.3s173.7 MiB [] 3% 7.7s173.7 MiB [] 3% 7.2s173.7 MiB [] 3% 6.8s173.7 MiB [] 4% 6.6s173.7 MiB [] 4% 6.3s173.7 MiB [] 4% 6.2s173.7 MiB [] 5% 6.1s173.7 MiB [] 5% 6.4s173.7 MiB [] 6% 6.3s173.7 MiB [] 7% 6.1s173.7 MiB [] 7% 6.0s173.7 MiB [] 7% 5.9s173.7 MiB [] 8% 5.7s173.7 MiB [] 8% 5.8s173.7 MiB [] 8% 5.9s173.7 MiB [] 9% 6.0s173.7 MiB [] 9% 5.9s173.7 MiB [] 9% 5.8s173.7 MiB [] 10% 5.7s173.7 MiB [] 10% 5.6s173.7 MiB [] 10% 5.7s173.7 MiB [] 11% 5.6s173.7 MiB [] 11% 5.4s173.7 MiB [] 12% 5.4s173.7 MiB [] 12% 5.3s173.7 MiB [] 13% 5.2s173.7 MiB [] 14% 5.1s173.7 MiB [] 14% 5.0s173.7 MiB [] 15% 4.9s173.7 MiB [] 16% 4.7s173.7 MiB [] 17% 4.5s173.7 MiB [] 17% 4.4s173.7 MiB [] 18% 4.4s173.7 MiB [] 18% 4.3s173.7 MiB [] 19

In [7]:
!pip install selenium webdriver-manager


In [31]:
!pip install playwright
!playwright install chromium


In [34]:
import asyncio
from playwright.async_api import async_playwright

async def main():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto("https://www.cars24.com/buy-used-kia-cars-mumbai/")

        # Get all Kia car links
        links = await page.query_selector_all("a.styles_carCardWrapper__sXLIp")
        car_links = [await link.get_attribute("href") for link in links if 'kia' in (await link.get_attribute("href")).lower()]

        print(f"Found {len(car_links)} Kia car links")
        await browser.close()

# Run the async function
await main()


Found 20 Kia car links


In [36]:
import asyncio
from playwright.async_api import async_playwright

async def get_all_kia_links():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto("https://www.cars24.com/buy-used-kia-cars-mumbai/")

        car_links = set()
        previous_count = 0
        retries = 0

        while retries < 5:  # stop if no new cars after 5 scrolls
            # Scroll down
            await page.evaluate("window.scrollBy(0, 1000)")
            await asyncio.sleep(2)  # wait for new cars to load

            links = await page.query_selector_all("a.styles_carCardWrapper__sXLIp")
            for link in links:
                href = await link.get_attribute("href")
                if href and 'kia' in href.lower():
                    car_links.add(href)

            if len(car_links) == previous_count:
                retries += 1
            else:
                retries = 0
                previous_count = len(car_links)

        print(f"Total Kia car links found: {len(car_links)}")
        await browser.close()
        return list(car_links)

# Run
all_links = await get_all_kia_links()


Total Kia car links found: 38


In [37]:
import asyncio
import re
import pandas as pd
from playwright.async_api import async_playwright

# -------------------------
# Utility Functions
# -------------------------
def clean_price_to_numerical(price_text):
    if not price_text:
        return None
    try:
        price_clean = price_text.replace("₹", "").replace("â‚¹", "").strip()
        match = re.search(r"([\d\.]+)\s*(lakh|crore|cr)?", price_clean.lower())
        if match:
            num = float(match.group(1))
            unit = match.group(2)
            if unit and "lakh" in unit:
                return int(num * 100000)
            elif unit and ("crore" in unit or "cr" in unit):
                return int(num * 10000000)
            else:
                return int(num)
        return None
    except:
        return None

def clean_owners_to_numerical(owner_text):
    if not owner_text:
        return None
    text = owner_text.lower()
    mappings = {
        '1st': 1, 'first': 1, '1': 1,
        '2nd': 2, 'second': 2, '2': 2,
        '3rd': 3, 'third': 3, '3': 3,
        '4th': 4, 'fourth': 4, '4': 4,
        '5th': 5, 'fifth': 5, '5': 5,
        '6th': 6, 'sixth': 6, '6': 6
    }
    for k, v in mappings.items():
        if k in text:
            return v
    m = re.search(r'(\d+)', text)
    if m:
        return min(int(m.group(1)), 6)
    return None

def clean_km(val):
    if not val:
        return None
    val = re.sub(r"[^\d]", "", str(val))
    return int(val) if val else None

# -------------------------
# Extract Details for a Car
# -------------------------
async def extract_car_details(page, link, city="Mumbai"):
    await page.goto(link)
    await page.wait_for_timeout(2000)  # wait for page load

    car = {"location": city, "brand": "KIA"}

    # Title and Year
    try:
        title = await page.text_content("h1")
        car["title"] = title
        parts = title.split()
        car["year"] = parts[0] if parts[0].isdigit() else None
    except:
        car["title"] = car["year"] = None

    # Price
    try:
        price_text = await page.text_content("p.sc-braxZu.hhzsvw")
        car["price_raw"] = price_text
        car["price_numerical"] = clean_price_to_numerical(price_text)
    except:
        car["price_raw"] = car["price_numerical"] = None

    # Specs: km, owners, transmission, fuel
    try:
        specs = await page.query_selector_all("p.kvfdZL")
        specs_list = [await s.text_content() for s in specs if (await s.text_content()).strip()]
        if len(specs_list) >= 4:
            car["km_driven"] = specs_list[0]
            car["owners_raw"] = specs_list[1]
            car["transmission"] = specs_list[2]
            car["fuel_type"] = specs_list[3]
            car["km_numerical"] = clean_km(specs_list[0])
            car["owners_num"] = clean_owners_to_numerical(specs_list[1])
    except:
        car["km_driven"] = car["owners_raw"] = car["transmission"] = car["fuel_type"] = None
        car["km_numerical"] = car["owners_num"] = None

    return car

# -------------------------
# Main Scraper
# -------------------------
async def main():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        # Get all Kia car links
        car_links = await get_all_kia_links()  # reuse previous scrolling function

        all_data = []
        for i, link in enumerate(car_links, 1):
            print(f"Processing {i}/{len(car_links)}")
            try:
                car = await extract_car_details(page, link)
                all_data.append(car)
            except Exception as e:
                print(f"Error processing {link}: {e}")

        await browser.close()

        # Save to CSV
        df = pd.DataFrame(all_data)
        df.to_csv("Kia_Mumbai_Async.csv", index=False)
        print("Saved CSV with", len(df), "cars")
        print(df.head())

# -------------------------
# Run in Colab
# -------------------------
await main()


Total Kia car links found: 38
Processing 1/38
Processing 2/38
Processing 3/38
Processing 4/38
Processing 5/38
Processing 6/38
Processing 7/38
Processing 8/38
Processing 9/38
Processing 10/38
Processing 11/38
Processing 12/38
Processing 13/38
Processing 14/38
Processing 15/38
Processing 16/38
Processing 17/38
Processing 18/38
Processing 19/38
Processing 20/38
Processing 21/38
Processing 22/38
Processing 23/38
Processing 24/38
Processing 25/38
Processing 26/38
Processing 27/38
Processing 28/38
Processing 29/38
Processing 30/38
Processing 31/38
Processing 32/38
Processing 33/38
Processing 34/38
Processing 35/38
Processing 36/38
Processing 37/38
Processing 38/38
Saved CSV with 38 cars
  location brand                                    title  year    price_raw  \
0   Mumbai   KIA           2020 KIA SELTOS HTX 1.5 PETROL  2020   ₹9.99 lakh   
1   Mumbai   KIA  2023 KIA SELTOS  HTK PLUS 1.5 PETROL MT  2023  ₹11.45 lakh   
2   Mumbai   KIA               2020 KIA SONET HTX 1.0 IMT  2020   ₹7.9